In [16]:
#pip install packages
!pip install datasets
!pip install sentence_transformers

In [17]:
#import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset #to load datasets from hf
from datasets import concatenate_datasets
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample, datasets, losses, SentenceTransformer, util, evaluation
from torch.utils.data import DataLoader

# Upload and Prepare Datasets

In [18]:
#upload med_wikidoc dataset
med_wikidoc = load_dataset("medalpaca/medical_meadow_wikidoc")

#remove unnecessary columns
med_wikidoc = med_wikidoc.remove_columns(['instruction'])

# concatenate splits
med_wikidoc = concatenate_datasets([med_wikidoc["train"]])

#rename column name
med_wikidoc = med_wikidoc.rename_column('input','question')
med_wikidoc = med_wikidoc.rename_column('output','answer')

med_wikidoc

Dataset({
    features: ['question', 'answer'],
    num_rows: 10000
})

In [19]:
#show an example of dataset
med_wikidoc[0]

{'question': "Can you provide an overview of the lung's squamous cell carcinoma?",
 'answer': 'Squamous cell carcinoma of the lung may be classified according to the WHO histological classification system into 4 main types: papillary, clear cell, small cell, and basaloid.'}

In [20]:
#upload meadow dataset
meadow_dataset = load_dataset("medalpaca/medical_meadow_medical_flashcards")

#remove unnecessary columns
meadow_dataset=meadow_dataset.remove_columns(['instruction'])

# concatenate splits
meadow_dataset = concatenate_datasets([meadow_dataset["train"]])

#rename column name
meadow_dataset = meadow_dataset.rename_column('input','question')
meadow_dataset = meadow_dataset.rename_column('output','answer')

meadow_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 33955
})

In [21]:
#show an example of dataset
meadow_dataset[0]

{'question': 'What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?',
 'answer': 'Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.'}

In [22]:
#upload medQuad dataset
MedQuad_dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

#remove unnecessary columns
MedQuad_dataset =MedQuad_dataset.remove_columns(['qtype'])

# concatenate splits
MedQuad_dataset = concatenate_datasets([MedQuad_dataset["train"]])

#rename column name
MedQuad_dataset = MedQuad_dataset.rename_column('Question','question')
MedQuad_dataset = MedQuad_dataset.rename_column('Answer','answer')

MedQuad_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 16407
})

In [23]:
#show an example of dataset
MedQuad_dataset[0]

{'question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?',
 'answer': 'LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'}

In [24]:
#upload meadow wikidoc patient dataset
patient_dataset = load_dataset("medalpaca/medical_meadow_wikidoc_patient_information")

#remove unnecessary columns
patient_dataset = patient_dataset.remove_columns(['instruction'])

# concatenate splits
patient_dataset = concatenate_datasets([patient_dataset["train"]])

#rename column name
patient_dataset = patient_dataset.rename_column('input','question')
patient_dataset = patient_dataset.rename_column('output','answer')

patient_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 5942
})

In [25]:
#show an example of dataset
patient_dataset[0]

{'question': 'What are the symptoms of Allergy?',
 'answer': 'Allergy symptoms vary, but may include:\nBreathing problems (coughing, shortness of breath) Burning, tearing, or itchy eyes Conjunctivitis (red, swollen eyes) Coughing Diarrhea Headache Hives Itching of the nose, mouth, throat, skin, or any other area Runny nose Skin rashes Stomach cramps Vomiting Wheezing\nWhat part of the body is contacted by the allergen plays a role in the symptoms you develop. For example:\nAllergens that are breathed in often cause a stuffy nose, itchy nose and throat, mucus production, cough, or wheezing. Allergens that touch the eyes may cause itchy, watery, red, swollen eyes. Eating something you are allergic to can cause nausea, vomiting, abdominal pain, cramping, diarrhea, or a severe, life-threatening reaction. Allergens that touch the skin can cause a skin rash, hives, itching, blisters, or even skin peeling. Drug allergies usually involve the whole body and can lead to a variety of symptoms.'}

In [26]:
# Concatenate all of the datasets
new_df = concatenate_datasets([patient_dataset,MedQuad_dataset,med_wikidoc,meadow_dataset])

# Verify the concatenated dataset
new_df

Dataset({
    features: ['question', 'answer'],
    num_rows: 66304
})

In [27]:
#train test split
new_df = new_df.train_test_split(test_size=0.2,shuffle=True)

print(f"len({new_df})")

len(DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 53043
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 13261
    })
}))


In [28]:
#define the model
model = SentenceTransformer('BAAI/bge-small-en-v1.5')
#change model's seq length 
model.max_seq_length = 128

model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [29]:
#create inputexample for train question answer pairs
train_data=[]
def create_train_inputexample(example):
    question = example['question']
    answer  = example['answer']
    # Create an InputExample object and append it to the data list
    input_example = InputExample(texts=[str(question),str(answer)])
    train_data.append(input_example)
    return None

new_df['train'].map(create_train_inputexample)

Map:   0%|          | 0/53043 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 53043
})

In [30]:
#verify the format
train_data[1]

In [31]:
#create lists for both test questions and answers
test_questions=[]
test_answers=[]
def create_test_inputexample(example):
    test_questions.append(example['question'])
    test_answers.append(example['answer'])
    return None

new_df['test'].map(create_test_inputexample)
#verify the lists
print(test_questions[1])
print(test_answers[1])

Map:   0%|          | 0/13261 [00:00<?, ? examples/s]

What to do for Porphyria ?
- Porphyrias are rare disorders that affect mainly the skin or nervous system and may cause abdominal pain.  - Each of the eight types of porphyria corresponds to low levels of a specific enzyme in the heme biosynthetic pathway.  - The exact rates of porphyria are unknown and vary around the world. For example, porphyria cutanea tarda is most common in the United States, and variegate porphyria is most common in South America.  - Most porphyrias result from inheriting an abnormal gene, also called a gene mutation, from one parent.  - Porphyria cutanea tarda is usually an acquired disorder, meaning factors other than genes cause the enzyme deficiency.  - Symptoms of cutaneous porphyrias include      - oversensitivity to sunlight    - blisters on exposed areas of the skin    - itching and swelling on exposed areas of the skin    - Symptoms of acute porphyrias include      - pain in the abdomen    - pain in the chest, limbs, or back    - nausea and vomiting    -

In [32]:
#define the evaluator
dev_evaluator = evaluation.TranslationEvaluator(test_questions,test_answers, show_progress_bar=True, write_csv=True,batch_size=64)

In [33]:
#define train loss and train loader
train_loss = losses.CachedMultipleNegativesRankingLoss(model,mini_batch_size=32)
train_loader = datasets.NoDuplicatesDataLoader(train_data, batch_size=512)

In [31]:
#evaluate the model
eval_initial = model.evaluate(dev_evaluator)
#print initial evaluation
print(f"Initial evaluation {eval_initial}")

Batches:   0%|          | 0/259 [00:00<?, ?it/s]

Batches:   0%|          | 0/259 [00:00<?, ?it/s]

Initial evaluation 0.6645936481101316


In [ ]:
#train the model
model.fit(train_objectives=[(train_loader, train_loss)],evaluation_steps=100,epochs=80, warmup_steps=100,
evaluator=dev_evaluator,use_amp=True,optimizer_params = {'lr': 24e-5},     
output_path="/kaggle/working/results/medical_qa")

In [ ]:
#save the model
model.save('/home/aleynahuk/medical_qa/bge-small-en-v1.5')